In [1]:
import pandas as pd
import os

In [2]:
pd.set_option('display.max_columns', None)
path = os.path.join(os.getcwd(), 'Data')
df = pd.read_csv(os.path.join(path, 'DelayedFlights.csv'))

In [3]:
df.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(1936758, 30)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 30 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   Year               int64  
 2   Month              int64  
 3   DayofMonth         int64  
 4   DayOfWeek          int64  
 5   DepTime            float64
 6   CRSDepTime         int64  
 7   ArrTime            float64
 8   CRSArrTime         int64  
 9   UniqueCarrier      object 
 10  FlightNum          int64  
 11  TailNum            object 
 12  ActualElapsedTime  float64
 13  CRSElapsedTime     float64
 14  AirTime            float64
 15  ArrDelay           float64
 16  DepDelay           float64
 17  Origin             object 
 18  Dest               object 
 19  Distance           int64  
 20  TaxiIn             float64
 21  TaxiOut            float64
 22  Cancelled          int64  
 23  CancellationCode   object 
 24  Diverted           int64  
 25  CarrierDelay      

In [6]:
# df['Cancelled'].value_counts()
cancelled = df['Cancelled'] == 1
airtime = df['AirTime'].notna()
df[cancelled & airtime]

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay


In [7]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip()

In [8]:
df = df[df['Cancelled'] == 0]

df = df.rename(columns={
    'TaxiIn': 'TaxiIn (time from landing to gate)',
    'TaxiOut': 'TaxiOut (time from gate to takeoff)',
    'Diverted': 'Diverted (Changed landing ap)',
    'TailNum': 'TailNum (Unique Aircraft identifier)',
    'UniqueCarrier' : 'UniqueCarrier (Airline)',
    'FlightNum':'FlightNum (Route Number)'})

df = df.drop(columns=['Unnamed: 0','DayOfWeek','Cancelled','CancellationCode'])
df.reset_index(drop=True, inplace=True)


In [9]:
# df['ActualElapsedTime'].min()
df['ActualElapsedTime'].max()

1114.0

In [10]:
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-' + df['DayofMonth'].astype(str)).dt.date
df = df[['Date'] + [c for c in df.columns if c != 'Date']]

In [11]:
def timeFunc(df, timeColumns):
    df = df.dropna(subset=timeColumns)
    for col in timeColumns:
        df[col] = df[col].astype(int).astype(str).replace('2400', '0000')
        df[col] = df[col].str.zfill(4)
        df[col] = pd.to_datetime(df[col], format='%H%M', errors='coerce').dt.time
    return df
    
# print([col for col in df.columns if 'Time' in col])
time_cols = ['DepTime',	'CRSDepTime',	'ArrTime',	'CRSArrTime']
df = timeFunc(df, time_cols)
print('done')

C:\Users\abdel\AppData\Local\Temp\ipykernel_772\2479240239.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(int).astype(str).replace('2400', '0000')
C:\Users\abdel\AppData\Local\Temp\ipykernel_772\2479240239.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.zfill(4)
C:\Users\abdel\AppData\Local\Temp\ipykernel_772\2479240239.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

done


C:\Users\abdel\AppData\Local\Temp\ipykernel_772\2479240239.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col], format='%H%M', errors='coerce').dt.time


In [12]:
df.head()

,Date,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier (Airline),FlightNum (Route Number),TailNum (Unique Aircraft identifier),ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn (time from landing to gate),TaxiOut (time from gate to takeoff),Diverted (Changed landing ap),CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03,2008,1,3,20:03:00,19:55:00,22:11:00,22:25:00,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,NaN,NaN,NaN,NaN
1,2008-01-03,2008,1,3,07:54:00,07:35:00,10:02:00,10:00:00,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,NaN,NaN,NaN,NaN
2,2008-01-03,2008,1,3,06:28:00,06:20:00,08:04:00,07:50:00,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,NaN,NaN,NaN,NaN
3,2008-01-03,2008,1,3,18:29:00,17:55:00,19:59:00,19:25:00,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,2.0,0.0,0.0,0.0,32.0
4,2008-01-03,2008,1,3,19:40:00,19:15:00,21:21:00,21:10:00,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,NaN,NaN,NaN,NaN,NaN


In [13]:
cols = ['ArrDelay', 'DepDelay', 'CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']
df[cols] = df[cols].fillna(0)


In [14]:
df.shape

(1929648, 27)

In [15]:
df.nunique()

Date                                     366
Year                                       1
Month                                     12
DayofMonth                                31
DepTime                                 1438
CRSDepTime                              1193
ArrTime                                 1440
CRSArrTime                              1361
UniqueCarrier (Airline)                   20
FlightNum (Route Number)                7498
TailNum (Unique Aircraft identifier)    5360
ActualElapsedTime                        673
CRSElapsedTime                           501
AirTime                                  650
ArrDelay                                1128
DepDelay                                1053
Origin                                   303
Dest                                     302
Distance                                1407
TaxiIn (time from landing to gate)       180
TaxiOut (time from gate to takeoff)      332
Diverted (Changed landing ap)              2
CarrierDel

In [16]:
df[df.duplicated(['UniqueCarrier (Airline)','FlightNum (Route Number)', 
               'Date', 'TailNum (Unique Aircraft identifier)'])].sort_values(
                by=['FlightNum (Route Number)','Date','TailNum (Unique Aircraft identifier)']).head(3)


,Date,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier (Airline),FlightNum (Route Number),TailNum (Unique Aircraft identifier),ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn (time from landing to gate),TaxiOut (time from gate to takeoff),Diverted (Changed landing ap),CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
168334,2008-01-01,2008,1,1,11:44:00,10:55:00,14:59:00,13:59:00,B6,1,N534JB,195.0,184.0,169.0,60.0,49.0,JFK,FLL,1069,3.0,23.0,0,34.0,0.0,11.0,0.0,15.0
166711,2008-01-18,2008,1,18,11:56:00,11:50:00,14:38:00,14:31:00,AS,1,N564AS,222.0,221.0,198.0,7.0,6.0,SEA,ANC,1449,4.0,20.0,0,0.0,0.0,0.0,0.0,0.0
19960,2008-01-22,2008,1,22,08:58:00,08:25:00,09:50:00,09:15:00,WN,1,N505SW,52.0,50.0,39.0,35.0,33.0,HOU,CRP,187,4.0,9.0,0,0.0,25.0,2.0,0.0,8.0


In [17]:
df[df.duplicated(['UniqueCarrier (Airline)','FlightNum (Route Number)','Origin','Dest'])].sort_values(
                by=['UniqueCarrier (Airline)','FlightNum (Route Number)','Date']).head(3)

,Date,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier (Airline),FlightNum (Route Number),TailNum (Unique Aircraft identifier),ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn (time from landing to gate),TaxiOut (time from gate to takeoff),Diverted (Changed landing ap),CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
138534,2008-01-06,2008,1,6,07:58:00,07:15:00,09:08:00,08:35:00,9E,2001,91479E,130.0,140.0,104.0,33.0,43.0,ATL,HOU,696,2.0,24.0,0,33.0,0.0,0.0,0.0,0.0
138535,2008-01-19,2008,1,19,07:28:00,07:15:00,08:38:00,08:35:00,9E,2001,91539E,130.0,140.0,117.0,3.0,13.0,ATL,HOU,696,1.0,12.0,0,0.0,0.0,0.0,0.0,0.0
138536,2008-01-20,2008,1,20,07:40:00,07:15:00,08:41:00,08:35:00,9E,2001,91469E,121.0,140.0,106.0,6.0,25.0,ATL,HOU,696,1.0,14.0,0,0.0,0.0,0.0,0.0,0.0


In [18]:
DimDate = df[['Date','Month','DayofMonth']].drop_duplicates().reset_index(drop=True)

In [19]:
df['TailNum (Unique Aircraft identifier)'].drop_duplicates().shape[0]

5361

In [20]:
df[['Origin','Dest','Distance']].drop_duplicates().shape[0]

5127

In [21]:
df[['UniqueCarrier (Airline)','Origin','Dest','Distance']].drop_duplicates().shape[0]

8321

In [22]:
DimRoute = df[['UniqueCarrier (Airline)','Origin','Dest','Distance']].drop_duplicates().reset_index(drop=True)
DimRoute.index.name = 'FlightNum'

In [23]:
DimAirCraft = df['TailNum (Unique Aircraft identifier)'].drop_duplicates().reset_index(drop=True)
DimAirCraft.index.name = 'AirCraftID'

In [24]:
FactFlights = df[['Date','DepTime','CRSDepTime','ArrTime', 'CRSArrTime', 'TailNum (Unique Aircraft identifier)','ActualElapsedTime',
                  'CRSElapsedTime','AirTime','ArrDelay','DepDelay','TaxiIn (time from landing to gate)','TaxiOut (time from gate to takeoff)',
                  'Diverted (Changed landing ap)', 'CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay',
                  'UniqueCarrier (Airline)', 'Origin', 'Dest', 'Distance']]
FactFlights = FactFlights.merge(
    DimRoute.reset_index(),
    on=['UniqueCarrier (Airline)', 'Origin', 'Dest', 'Distance'],
    how='left')
FactFlights = FactFlights.merge(
    DimAirCraft.reset_index(),
    on=['TailNum (Unique Aircraft identifier)'],
    how='left')
FactFlights.drop(columns=['Origin', 'Dest', 'Distance','UniqueCarrier (Airline)','TailNum (Unique Aircraft identifier)'],inplace=True)

In [25]:
FactFlights = FactFlights[['Date', 'FlightNum', 'AirCraftID'] + 
                          [col for col in FactFlights.columns if col not in ['Date','FlightNum','AirCraftID']]]

In [26]:
FactFlights.head()

,Date,FlightNum,AirCraftID,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,TaxiIn (time from landing to gate),TaxiOut (time from gate to takeoff),Diverted (Changed landing ap),CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03,0,0,20:03:00,19:55:00,22:11:00,22:25:00,128.0,150.0,116.0,-14.0,8.0,4.0,8.0,0,0.0,0.0,0.0,0.0,0.0
1,2008-01-03,0,1,07:54:00,07:35:00,10:02:00,10:00:00,128.0,145.0,113.0,2.0,19.0,5.0,10.0,0,0.0,0.0,0.0,0.0,0.0
2,2008-01-03,1,2,06:28:00,06:20:00,08:04:00,07:50:00,96.0,90.0,76.0,14.0,8.0,3.0,17.0,0,0.0,0.0,0.0,0.0,0.0
3,2008-01-03,1,3,18:29:00,17:55:00,19:59:00,19:25:00,90.0,90.0,77.0,34.0,34.0,3.0,10.0,0,2.0,0.0,0.0,0.0,32.0
4,2008-01-03,2,4,19:40:00,19:15:00,21:21:00,21:10:00,101.0,115.0,87.0,11.0,25.0,4.0,10.0,0,0.0,0.0,0.0,0.0,0.0


In [27]:
DimAirCraft.head()

AirCraftID
0    N712SW
1    N772SW
2    N428WN
3    N464WN
4    N726SW
Name: TailNum (Unique Aircraft identifier), dtype: object

In [28]:
DimRoute.head()

,UniqueCarrier (Airline),Origin,Dest,Distance
FlightNum,,,,
0,WN,IAD,TPA,810
1,WN,IND,BWI,515
2,WN,IND,JAX,688
3,WN,IND,LAS,1591
4,WN,IND,MCO,828


In [29]:
DimDate.head()

,Date,Month,DayofMonth
0,2008-01-03,1,3
1,2008-01-04,1,4
2,2008-01-05,1,5
3,2008-01-06,1,6
4,2008-01-07,1,7


In [30]:
model = os.path.join(path, 'Model')
os.makedirs(model, exist_ok=True)

FactFlights.to_csv(os.path.join(model, 'FactFlights.csv'), index=False)
DimRoute.to_csv(os.path.join(model, 'DimRoute.csv'))
DimAirCraft.to_csv(os.path.join(model, 'DimAirCraft.csv'))
DimDate.to_csv(os.path.join(model, 'DimDate.csv'), index=False)